## Simulation of one round of blockchain-based federated learning 

In [3]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [4]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available and set the default tensor type to CUDA
print('Using device: %s' % device)
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

Cuda available: True
NVIDIA GeForce MX150
major and minor cuda capability of the device: (6, 1)
Using device: cuda
Cuda set as default device


### Config variables for models and training


In [5]:
from os.path import join as pjoin
import json
from client.train import train
from client.model.perceptron import MultiLayerPerceptron
from client.services.gateway_client import submit_local_model, get_all_models, get_model
from client.utils import load_model_from_json, weights_zero_init,  weights_init, update_lr
from client.aggregators import federated_aggregate
from client.dataloader import get_cifar10_dataloaders, get_cifar10_datasets

input_size = 32 * 32 * 3
hidden_size = [50]
num_classes = 10
num_epochs = 1
learning_rate = 1e-3
learning_rate_decay = 0.95
reg = 0.001
modelpath = 'client/models/'
train_flag = True


### Load data


In [6]:

root = 'client/data/'
num_training = 49000
num_validation = 1000
batch_size = 200
train_dataset, val_dataset, test_dataset = get_cifar10_datasets(
    root, num_training, num_validation)
train_loader, val_loader, test_loader = get_cifar10_dataloaders(
    root, batch_size, num_training, num_validation, device)

dataloaders = {
    'train': train_loader,
    'validation': val_loader,
    'test': test_loader
}


Files already downloaded and verified
Files already downloaded and verified


### Create two local models


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron

model1 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model1.to(device)
print(model1)

model2 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model2.to(device)
print(model2)



MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)


### Train first local model


In [8]:
model1_name = 'bcfl_model3'

# Training
model1.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model1.parameters(), lr=learning_rate, weight_decay=reg)

train(model1, modelpath, model1_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)



Epoch [1/1], Step [1/245], Loss: 2.3026
Epoch [1/1], Step [2/245], Loss: 2.3009
Epoch [1/1], Step [3/245], Loss: 2.2944
Epoch [1/1], Step [4/245], Loss: 2.2815
Epoch [1/1], Step [5/245], Loss: 2.2652
Epoch [1/1], Step [6/245], Loss: 2.2571
Epoch [1/1], Step [7/245], Loss: 2.2337
Epoch [1/1], Step [8/245], Loss: 2.2261
Epoch [1/1], Step [9/245], Loss: 2.2200
Epoch [1/1], Step [10/245], Loss: 2.1710
Epoch [1/1], Step [11/245], Loss: 2.1556
Epoch [1/1], Step [12/245], Loss: 2.1138
Epoch [1/1], Step [13/245], Loss: 2.0962
Epoch [1/1], Step [14/245], Loss: 2.1602
Epoch [1/1], Step [15/245], Loss: 2.0818
Epoch [1/1], Step [16/245], Loss: 2.1103
Epoch [1/1], Step [17/245], Loss: 1.9963
Epoch [1/1], Step [18/245], Loss: 1.9784
Epoch [1/1], Step [19/245], Loss: 2.0129
Epoch [1/1], Step [20/245], Loss: 2.0633
Epoch [1/1], Step [21/245], Loss: 1.9089
Epoch [1/1], Step [22/245], Loss: 1.9906
Epoch [1/1], Step [23/245], Loss: 2.0812
Epoch [1/1], Step [24/245], Loss: 1.8901
Epoch [1/1], Step [25/245

### Train second local model

In [9]:
model2_name = 'bcfl_model4'
# Training
model2.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model2.parameters(), lr=learning_rate, weight_decay=reg)

train(model2, modelpath, model2_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)

Epoch [1/1], Step [1/245], Loss: 2.3026
Epoch [1/1], Step [2/245], Loss: 2.3006
Epoch [1/1], Step [3/245], Loss: 2.2943
Epoch [1/1], Step [4/245], Loss: 2.2900
Epoch [1/1], Step [5/245], Loss: 2.2674
Epoch [1/1], Step [6/245], Loss: 2.2441
Epoch [1/1], Step [7/245], Loss: 2.2341
Epoch [1/1], Step [8/245], Loss: 2.2066
Epoch [1/1], Step [9/245], Loss: 2.1784
Epoch [1/1], Step [10/245], Loss: 2.1761
Epoch [1/1], Step [11/245], Loss: 2.1666
Epoch [1/1], Step [12/245], Loss: 2.1576
Epoch [1/1], Step [13/245], Loss: 2.0994
Epoch [1/1], Step [14/245], Loss: 2.1363
Epoch [1/1], Step [15/245], Loss: 2.0135
Epoch [1/1], Step [16/245], Loss: 2.0541
Epoch [1/1], Step [17/245], Loss: 2.1163
Epoch [1/1], Step [18/245], Loss: 2.0126
Epoch [1/1], Step [19/245], Loss: 2.0440
Epoch [1/1], Step [20/245], Loss: 2.0545
Epoch [1/1], Step [21/245], Loss: 2.0066
Epoch [1/1], Step [22/245], Loss: 2.0536
Epoch [1/1], Step [23/245], Loss: 2.0547
Epoch [1/1], Step [24/245], Loss: 1.9864
Epoch [1/1], Step [25/245

### Test the trained models

In [10]:
from os.path import join as pjoin
modelpath = 'client/models/'

# Run the test code once you have your by setting train flag to false
# and loading the best model
model_ckpt = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model_ckpt = torch.load(pjoin(modelpath, model1_name + '.ckpt'))
model1.load_state_dict(model_ckpt)
model = model1
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))
    
    

model = model2
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))

Accuracy of the network on the 1000 test images: 41.7 %
Accuracy of the network on the 1000 test images: 42.9 %



### Forward the two local models to the blockchain


In [12]:
# Send the saved model to Fabric-SDK via Gateway Client (REST call)

from client.services.gateway_client import submit_local_model, get_all_models, get_model


print(submit_local_model(model1_name, model1.state_dict()))

print(submit_local_model(model2_name, model2.state_dict()))



{'message': 'Error creating model', 'error': '10 ABORTED: failed to endorse transaction, see attached details for more info'}
{'message': 'Model submitted succesfully'}



### Aggregate the local models on the blockchain
TODO this should be triggered automatically when the total users in the blockchain have triggered their updates


In [14]:
# TODO this should be triggered automatically when the total users in the blockchain have triggered their updates

from client.services.gateway_client import aggregate_models
from client.utils import weights_zero_init

model3 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model3.to(device)
weights_zero_init(model3)
model3_name = 'bcfl_model_empty'
# print(model3)
print(submit_local_model(model3_name, model3.state_dict()))


aggregate_models([model1_name, model3_name])


{'message': 'Model submitted succesfully'}


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


### Download the new global model


In [13]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_local_model, get_all_models, get_model
from client.utils import load_model_from_json

# retrieve future global model and convert it again to pytorch model

# TODO change
model_avg_name = model1_name

model_params = get_model(model_avg_name)

global_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
global_model.to(device)
print(global_model)

load_model_from_json(global_model, model_params)
print(model)
   


MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)


### Print the new global model


In [14]:

print(model1.state_dict()['layers.0.bias'])
print(global_model.state_dict()['layers.0.bias'])
# the global model should be the same as model 1

tensor([ 0.0781,  0.0531,  0.0765,  0.0695,  0.0584,  0.0888, -0.0152, -0.0041,
         0.0655,  0.0632,  0.0207, -0.0331, -0.0308,  0.0388,  0.0202,  0.0874,
        -0.0027,  0.0364,  0.0482,  0.0680, -0.0027, -0.0260,  0.0715,  0.0244,
         0.0654,  0.0858,  0.0561,  0.0563, -0.0233,  0.0454,  0.0577, -0.0090,
        -0.0112, -0.0342, -0.0198, -0.0160,  0.0385, -0.0171,  0.0479,  0.0897,
         0.0603,  0.0571,  0.0074,  0.0662, -0.0064,  0.0493, -0.0284,  0.0637,
         0.0816,  0.0661], device='cuda:0')
tensor([ 0.0781,  0.0531,  0.0765,  0.0695,  0.0584,  0.0888, -0.0152, -0.0041,
         0.0655,  0.0632,  0.0207, -0.0331, -0.0308,  0.0388,  0.0202,  0.0874,
        -0.0027,  0.0364,  0.0482,  0.0680, -0.0027, -0.0260,  0.0715,  0.0244,
         0.0654,  0.0858,  0.0561,  0.0563, -0.0233,  0.0454,  0.0577, -0.0090,
        -0.0112, -0.0342, -0.0198, -0.0160,  0.0385, -0.0171,  0.0479,  0.0897,
         0.0603,  0.0571,  0.0074,  0.0662, -0.0064,  0.0493, -0.0284,  0.06

### Test the new global model

In [15]:

model = global_model
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))

Accuracy of the network on the 1000 test images: 41.8 %
